In [61]:
import gensim
import os
import collections
import random
import pandas as pd
import numpy as np
from stemming import porter2

In [3]:
data = pd.read_csv('/home/abhinav/dev/generate_man_corpus/man.csv', delimiter='\t')
data.data = data.data.apply(str)

In [2]:
ignore_list = set(['c', 'a.out', 'c++', 'md5', 'sha1', 'sha2', 'sha256', 'sha512', 'rc.d', 'rc.conf', 'man.conf', 'mk.conf'])

def remove_special_chars(word):
    if word in ignore_list:
        return word
    
    if word[0] == '-':
        return word

    if '%' in word:
        return None
    if '^' in word:
        return None
    if '&' in word:
        return None
    if '*' in word:
        return None
    if '[' in word or ']' in word:
        return None
    if '\\' in word:
        return None
    if '/' in word:
        return None
    if '|' in word:
        return None
    if '<' in word or '>' in word:
        return None
    if '~' in word:
        return None
    if '@' in word:
        return None
    if '=' in word:
        return None
    if '+' in word:
        return None
    if ':' in word:
        return None
    if '$' in word:
        return None
    if '}' in word or '{' in word:
        return None


    w = word.replace('-', ' ')
    w = w.replace('!', '')
    w = w.replace('#', '')
    w = w.replace('(', '')
    w = w.replace(')', '')
    w = w.replace(',', ' ')
    w = w.replace(';', ' ')
    w = w.replace('"', '')
    w = w.replace('`', '')
    w = w.replace('1', '')
    w = w.replace('2', '')
    w = w.replace('3', '')
    w = w.replace('4', '')
    w = w.replace('5', '')
    w = w.replace('6', '')
    w = w.replace('7', '')
    w = w.replace('8', '')
    w = w.replace('9', '')
    w = w.replace('0', '')
    w = w.replace('?', '')
    if '.' in w:
        if w[-1] == '.':
            w = w.replace('.', '')
        else:
            w = w.replace('.', ' ')

    if "'" in w:
        if len(w) > 2 and  w[-2] == "'" and w[-1] == 's':
            w = w[:-2]
        elif len(w) > 3 and w[-3] == "'" and w[-2] == 'l' and w[-1] == 'l':
            w = w[:-3]
        w = w.replace("'", '')
    return w


In [271]:
otexts = [document for document in data['data']]

texts = []
for i, document in enumerate(otexts):
    texts.append([])
    for word in document.split():
        r = remove_special_chars(word.lower())
        if r is None:
            continue
        texts[i].append((r))


In [272]:
docs = []
for i, text in enumerate(texts):
    docs.append(gensim.models.doc2vec.TaggedDocument(text, [i]))
    
    

In [317]:
model = gensim.models.doc2vec.Doc2Vec(size=150, min_count=5, iter=25, workers=4, window=8, dm_mean=1, dbow_words=1 )

In [318]:
train_corpus = docs


In [319]:
model.build_vocab(train_corpus)

In [320]:
%time model.train(train_corpus)

CPU times: user 31min 46s, sys: 2.32 s, total: 31min 48s
Wall time: 8min 2s


182836775

In [321]:
def get_most_similar(name, section=None):
    if section is None:
        index = np.where(data['name'] == name)[0][0]
    else:
        index = np.where((data['name'] == name) & (data['section'] == section))[0][0]
        
    inferred = model.infer_vector(train_corpus[index].words)
    sims = model.docvecs.most_similar([inferred], topn=10)
    for i,score in sims:
        print(data.name.iloc[i], score)


In [325]:
get_most_similar('ls')

ls 0.8469443917274475
du 0.49439680576324463
mtree 0.4322114586830139
size 0.41158851981163025
readlink 0.40169280767440796
fgrep 0.3958299160003662
grep 0.3953135311603546
stat 0.385476291179657
fprintf 0.385062038898468
vsnprintf 0.3821207582950592


In [326]:
def search(query):
    inferred = model.infer_vector([(word) for word in query.lower().split()])
    sims = model.docvecs.most_similar([inferred], topn=20)
    for i,score in sims:
        print(data.name.iloc[i], score)


In [334]:
search('open file')

csin 0.2766682505607605
extattr_list_fd 0.26679831743240356
cexpf 0.2654956579208374
ohci 0.25898098945617676
extattr_delete_link 0.2576597332954407
elf_kind 0.25668901205062866
extattr_list_file 0.2552754282951355
creal 0.2526130676269531
extattr_get_link 0.2510366439819336
extattr_set_link 0.2505379319190979
extattr_delete_fd 0.24676385521888733
dofilewritev 0.24390479922294617
exit 0.24350549280643463
cexp 0.23913246393203735
attimer 0.23768705129623413
boot 0.23649990558624268
closef 0.23555658757686615
cbb 0.23534046113491058
dofilewrite 0.2342350333929062
fdrelease 0.2337464988231659


In [330]:
model.save('./man.doc2vec')